# Idealized entrainment

This notebook runs [GOTM](https://gotm.net/) using the [entrainment](https://gotm.net/cases/entrainment/) test case, an idealized wind stress-driven entrainment case with no rotation, in which the mixed layer gradually entrains into an underlying non-turbulent region with constant stable stratification.

Four turbulence closure schemes are used:

- GLS-C01A: The generic length scale ([Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).

- Three variants of KPP via [CVMix](http://cvmix.github.io):
    - KPP-CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf))
    - KPPLT-VR12 ([Li et al., 2016](https://doi.org/10.1016%2Fj.ocemod.2015.07.020))
    - KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
    
Run the case with a combination of vertical resolution of [0.2, 1., 5.] m and time step of [6, 60, 600] s.

In [1]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='Entrainment', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/code
   gotmdir_data: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/data
  gotmdir_build: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/build
    gotmdir_exe: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/exe
    gotmdir_run: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run
 gotmdir_figure: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/figure
   gotmdir_tool: /Users/qingli/work/A2020_CVMix_in_GOTM/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 3.09 ms, sys: 9.25 ms, total: 12.3 ms
Wall time: 82.1 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run/Entrainment/gotm.yaml'...
Done!


Update the configuration

In [6]:
# setup
title = 'Shear-driven Entrainment'
depth = 50.0
cfg['title'] = title
cfg['location']['name'] = 'equator'
cfg['location']['latitude'] = 0.0
cfg['location']['longitude'] = 0.0
cfg['location']['depth'] = depth
cfg['time']['start'] = '2005-01-01 00:00:00'
cfg['time']['stop']  = '2005-01-02 06:00:00'

# output
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['time_unit'] = 'second'
cfg['output']['gotm_out']['time_step'] = 1800

# forcing
cfg['temperature']['method'] = 'buoyancy'
cfg['temperature']['two_layer']['t_s'] = 20.0
cfg['temperature']['NN'] = 1e-4
cfg['salinity']['method'] = 'constant'
cfg['salinity']['constant_value'] = 35.0
cfg['surface']['fluxes']['method'] = 'off'
cfg['surface']['fluxes']['heat']['method'] = 'constant'
# since KPPLT-LF17 only use the Langmuir enhanced entrainment
# under destabilizing conditions, use a small destabilizing heat
# flux to activate it
cfg['surface']['fluxes']['heat']['constant_value'] = -1.0e-12 
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['constant_value'] = 1.0e-1
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0

cfg['waves']['stokes_drift']['us']['method'] = 'exponential'
cfg['waves']['stokes_drift']['vs']['method'] = 'exponential'
cfg['waves']['stokes_drift']['exponential']['us0']['method'] = 'constant'
cfg['waves']['stokes_drift']['exponential']['us0']['constant_value'] = 0.111
cfg['waves']['stokes_drift']['exponential']['vs0']['method'] = 'constant'
cfg['waves']['stokes_drift']['exponential']['vs0']['constant_value'] = 0.0
cfg['waves']['stokes_drift']['exponential']['ds']['method'] = 'constant'
cfg['waves']['stokes_drift']['exponential']['ds']['constant_value'] = 5.0

# EOS 
cfg['eq_state']['method'] = 'unesco'
cfg['eq_state']['form'] = 'full-pot'

# buoyancy
cfg['buoyancy']['NN_ini'] = 1.0e-4

# configure GLS-C01A
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

## Run the model

Set the configurations and labels for each run

In [7]:
cfgs = []
labels = []

In [8]:
for dz in [0.2, 1., 5.]:
    nlev = int(depth/dz)
    cfg['grid']['nlev']  = nlev
    cfg['output']['gotm_out']['k1_stop'] = nlev+1
    cfg['output']['gotm_out']['k_stop'] = nlev
    for dt in [6.0, 60.0, 600.0]:
        nt = int(30.*3600/dt)
        cfg['time']['dt'] = dt
        label = '_Nz{:d}_Nt{:d}'.format(nlev, nt)
        cfg['turbulence']['turb_method'] = 'second_order'
        cfgs.append(copy.deepcopy(cfg))
        labels.append('GLS-C01A'+label)
        cfg['turbulence']['turb_method'] = 'cvmix'
        cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
        cfgs.append(copy.deepcopy(cfg))
        labels.append('KPP-CVMix'+label)
        cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lwf16'
        cfgs.append(copy.deepcopy(cfg))
        labels.append('KPPLT-VR12'+label)
        cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lf17'
        cfgs.append(copy.deepcopy(cfg))
        labels.append('KPPLT-LF17'+label)

Run the cases in parallel with 2 processes 

In [9]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 61 ms, sys: 24.1 ms, total: 85.1 ms
Wall time: 12.3 s
